# 🎧 QualiVault: Prepare Audio
**Goal:** Convert raw audio files to 16-bit 16kHz FLAC (ideal for Whisper) and verify quality.

1. Reads `processing_recipe.yaml`.
2. Converts/Merges audio files using `ffmpeg`.
3. Saves them to the configured `flac_output_folder` (outside the git repo).
4. Provides an audio player to check quality and identify speakers.

In [ ]:
import yaml
from pathlib import Path
import IPython.display as ipd
from qualivault.audio import prepare_audio

# 1. Load Configuration
project_root = Path("..").resolve()
config_path = project_root / "config.yml"

with open(config_path) as f:
    config = yaml.safe_load(f)

# 2. Setup Output Folder
# Resolve path relative to project root
output_dir = (project_root / config['paths']['flac_output_folder']).resolve()
output_dir.mkdir(parents=True, exist_ok=True)
print(f"📂 FLAC Output Directory: {output_dir}")

In [ ]:
# 3. Load Recipe
recipe_path = project_root / "processing_recipe.yaml"
if not recipe_path.exists():
    print("❌ Recipe not found! Please run '00_Setup_and_Scan.ipynb' first.")
    recipe = []
else:
    with open(recipe_path) as f:
        recipe = yaml.safe_load(f)
    print(f"✅ Loaded recipe with {len(recipe)} interviews.")

In [ ]:
# 4. Process Audio
for item in recipe:
    i_id = item['id']
    input_files = item['files']
    output_name = item['output_name']
    output_path = output_dir / output_name
    
    print(f"\n🎧 Processing Interview: {i_id}")
    
    if output_path.exists():
        print(f"  ✅ File already exists: {output_path.name}")
    else:
        try:
            prepare_audio(input_files, output_path)
            print(f"  ✅ Converted to FLAC: {output_path.name}")
        except Exception as e:
            print(f"  ❌ Error: {e}")
            continue
            
    # 5. Quality Check (Audio Player)
    print("  ▶️  Listen to verify quality & speakers:")
    display(ipd.Audio(filename=output_path))